# Project1
## Momentum Trading
### MACD

In [1]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
#from MCForecastTools import MCSimulation
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import hvplot.pandas


%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

### Collect Apple and SP500 Prices Using the `requests` Library

In [3]:
# Set current amount of stock assets
my_apple_1time=1
my_sp500_1time=1

In [4]:
# stock API URLs
my_apple_1time_url = "https://api.alternative.me/v2/ticker/?symbol=AAPL"
my_sp500_1time_url = "https://api.alternative.me/v2/ticker/?symbol=SP500"

In [5]:
# Fetch current Apple price
apple_price=requests.get(my_apple_1time_url).json()
apple_price

# Fetch current SP500 price
sp500_price=requests.get(my_sp500_1time_url).json()
sp500_price

# Compute current value of my crpto
my_apple_value=my_apple_1time*apple_price['data']['1']['quotes']['USD']['price']
my_sp500_value=my_sp500_1time*sp500_price['data']['1027']['quotes']['USD']['price']

# Print current crypto wallet balance
print(f"The current value of your {my_apple_1time} AAPLE is ${my_apple_value:0.2f}")
print(f"The current value of your {my_sp500_1time} SP500 is ${my_sp500_value:0.2f}")

The current value of your 1 AAPLE is $34428.00
The current value of your 1 SP500 is $1795.91


### Collect Investments Data Using Alpaca: `AAPLE` (stocks) and `SP500` (stocks)

In [6]:
# Set current amount of shares
my_apple = 1
my_sp500 = 1

In [7]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [8]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"

# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_tickers.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.90,169.00,168.60,391,168.98,15303,168.871369,AAPL
2023-10-30 08:15:00+00:00,168.85,169.00,168.85,105,168.99,5249,168.955731,AAPL
2023-10-30 08:30:00+00:00,168.85,168.92,168.85,72,168.85,3911,168.863544,AAPL
2023-10-30 08:45:00+00:00,168.99,168.99,168.71,81,168.86,4184,168.869565,AAPL
2023-10-30 09:00:00+00:00,169.23,169.23,168.97,111,168.97,5241,169.137236,AAPL


In [9]:
# Reorganize the DataFrame
# Separate ticker data
df_tickers.drop(['high','low','trade_count','open','volume','vwap','symbol'], axis=1, inplace=True)

df_tickers.head()

,close
timestamp,
2023-10-30 08:00:00+00:00,168.90
2023-10-30 08:15:00+00:00,168.85
2023-10-30 08:30:00+00:00,168.85
2023-10-30 08:45:00+00:00,168.99
2023-10-30 09:00:00+00:00,169.23


In [10]:
df_tickers['return']=df_tickers['close'].pct_change()
df_tickers

,close,return
timestamp,,
2023-10-30 08:00:00+00:00,168.9000,NaN
2023-10-30 08:15:00+00:00,168.8500,-0.000296
2023-10-30 08:30:00+00:00,168.8500,0.000000
2023-10-30 08:45:00+00:00,168.9900,0.000829
2023-10-30 09:00:00+00:00,169.2300,0.001420
...,...,...
2023-10-30 22:45:00+00:00,169.8699,-0.000177
2023-10-30 23:00:00+00:00,169.8200,-0.000294
2023-10-30 23:15:00+00:00,169.8500,0.000177


### MACD formula

In [11]:
# Set MACD formula and iterate over Apple data for the specified intervals and period of time

moving_average_convergance_divergence_ema12 = df_tickers['return'].ewm(halflife=12).mean()
moving_average_convergance_divergence_ema26 = df_tickers['return'].ewm(halflife=26).mean()
  

In [12]:

MACD_line = moving_average_convergance_divergence_ema12 - moving_average_convergance_divergence_ema26
MACD_line.head()


timestamp
2023-10-30 08:00:00+00:00         NaN
2023-10-30 08:15:00+00:00    0.000000
2023-10-30 08:30:00+00:00    0.000002
2023-10-30 08:45:00+00:00    0.000012
2023-10-30 09:00:00+00:00    0.000023
Name: return, dtype: float64

In [13]:
Signal_line = MACD_line.ewm(halflife=9).mean()
Signal_line.head()

timestamp
2023-10-30 08:00:00+00:00         NaN
2023-10-30 08:15:00+00:00    0.000000
2023-10-30 08:30:00+00:00    0.000001
2023-10-30 08:45:00+00:00    0.000005
2023-10-30 09:00:00+00:00    0.000010
Name: return, dtype: float64

In [14]:
df_tickers=df_tickers.assign(ewm12=df_tickers['return'].ewm(halflife=12).mean())
df_tickers.head()

,close,return,ewm12
timestamp,,,
2023-10-30 08:00:00+00:00,168.90,NaN,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296,-0.000296
2023-10-30 08:30:00+00:00,168.85,0.000000,-0.000144
2023-10-30 08:45:00+00:00,168.99,0.000829,0.000199
2023-10-30 09:00:00+00:00,169.23,0.001420,0.000532


In [15]:
df_tickers=df_tickers.assign(ewm26=df_tickers['return'].ewm(halflife=26).mean())
df_tickers.head()

,close,return,ewm12,ewm26
timestamp,,,,
2023-10-30 08:00:00+00:00,168.90,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296,-0.000296,-0.000296
2023-10-30 08:30:00+00:00,168.85,0.000000,-0.000144,-0.000146
2023-10-30 08:45:00+00:00,168.99,0.000829,0.000199,0.000188
2023-10-30 09:00:00+00:00,169.23,0.001420,0.000532,0.000508


In [16]:
df_tickers=df_tickers.assign(macd=MACD_line)
df_tickers.head()

,close,return,ewm12,ewm26,macd
timestamp,,,,,
2023-10-30 08:00:00+00:00,168.90,NaN,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296,-0.000296,-0.000296,0.000000
2023-10-30 08:30:00+00:00,168.85,0.000000,-0.000144,-0.000146,0.000002
2023-10-30 08:45:00+00:00,168.99,0.000829,0.000199,0.000188,0.000012
2023-10-30 09:00:00+00:00,169.23,0.001420,0.000532,0.000508,0.000023


In [17]:
df_tickers=df_tickers.assign(signal=Signal_line)
df_tickers.head()

,close,return,ewm12,ewm26,macd,signal
timestamp,,,,,,
2023-10-30 08:00:00+00:00,168.90,NaN,NaN,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296,-0.000296,-0.000296,0.000000,0.000000
2023-10-30 08:30:00+00:00,168.85,0.000000,-0.000144,-0.000146,0.000002,0.000001
2023-10-30 08:45:00+00:00,168.99,0.000829,0.000199,0.000188,0.000012,0.000005
2023-10-30 09:00:00+00:00,169.23,0.001420,0.000532,0.000508,0.000023,0.000010


In [41]:
df_tickers=df_tickers.assign(histogram=MACD_line-Signal_line)
df_tickers.head()

,close,return,ewm12,ewm26,macd,signal,histogram
timestamp,,,,,,,
2023-10-30 08:00:00+00:00,168.90,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296,-0.000296,-0.000296,0.000000,0.000000,0.000000
2023-10-30 08:30:00+00:00,168.85,0.000000,-0.000144,-0.000146,0.000002,0.000001,0.000001
2023-10-30 08:45:00+00:00,168.99,0.000829,0.000199,0.000188,0.000012,0.000005,0.000007
2023-10-30 09:00:00+00:00,169.23,0.001420,0.000532,0.000508,0.000023,0.000010,0.000013


In [20]:
MACD_plot = df_tickers['macd'].hvplot.line(    
    x='timestamp', 
    y='macd',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [21]:
Signal_plot = df_tickers['signal'].hvplot.line(    
    x='timestamp', 
    y='signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [22]:
plot = MACD_plot * Signal_plot 

plot

:Overlay
   .Curve.Macd   :Curve   [timestamp]   (macd)
   .Curve.Signal :Curve   [timestamp]   (signal)

In [47]:
Histogram_plot = df_tickers['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Histogram_plot

:Bars   [timestamp]   (histogram)

In [23]:
# itterating over MACD line and Signal Line values of the same date in the final Data Frame above

buy=0
sell=0

for index, row in df_tickers.iterrows():
    date=row[0]
    macd=row['macd']
    signal=row['signal']
    returns=row['return']
    if macd > signal:
        buy+=returns
    elif signal > macd:
        sell+=returns    


# Calculate MACD results
if sell>buy:
    print(f"Congratulations, your MACD trading algorithm made a profit of ${buy}")
elif buy>sell:
    print(f"Unfortunately, your MACD trading algorithm suffered losses of ${sell}") 

Unfortunately, your MACD trading algorithm suffered losses of $-0.019855465453849974
